# ResNet (Tensorflow)
# 1) ResNet 기본 블록 구성하기

Tensorflow를 사용해 ResNet-34, ResNet-50을 구현해봅니다.

![ResNet](https://user-images.githubusercontent.com/48689553/146134640-ae78885a-a864-4761-8fc4-a970cea4906e.png)

- $O$: Size of output image
- $I$: Size of input image
- $K$: Size of kernel used in the Conv layer
- $N$: Number of kernels
- $S$: Stride of the convolution operation
- $P$: Padding size


$$
O = \frac{I-K+2P}{S} + 1
$$

In [4]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

In [24]:
import urllib3
urllib3.disable_warnings()

# tfds.disable_progress_bar()   # 이 주석을 풀면 데이터셋 다운로드과정의 프로그레스바가 나타나지 않습니다.

(ds_train, ds_test), ds_info = tfds.load(
    'cats_vs_dogs',
    split=('train[:80%]', 'train[80%:]'), # train으로만 구성된 데이터 셋으로 80%기준 분할
    shuffle_files=True,
    with_info=True,
    as_supervised=True) # as_supervised False인 경우 라벨 인수가 누락되었다는 TypeError:발생

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling cats_vs_dogs-train.tfrecord...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Dataset cats_vs_dogs downloaded and prepared to /aiffel/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


In [21]:
# Input Normalization

def normalize_and_resize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    image = tf.image.resize(image, (224,224))
    return tf.cast(image, tf.float32) / 255., label

def apply_normalize_on_dataset(ds, is_test=False, batch_size=16):
    ds = ds.map(
        normalize_and_resize_img, 
        num_parallel_calls=1
    )
    ds = ds.batch(batch_size)
    if not is_test:
        ds = ds.repeat()
        ds = ds.shuffle(200)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [5]:
def conv_block(input_layer,
              channel,
              kernel_size,
              strides=1,
              activation='relu'):
    
    x = keras.layers.Conv2D(filters=channel,
                           kernel_size=kernel_size,
                           kernel_initializer='he_normal',
                           padding='same',
                           strides=strides)(input_layer)
    
    if activation:
        x = keras.layers.Activation(activation)(x)
        
    return x

In [9]:
def build_resnet_block(input_layer,
                      num_cnn=3,
                      channel=64,
                      block_num=0,
                      resnet_50=True):
    # 입력 레이어
    x = input_layer
    
    # ResNet-50
    if resnet_50:
        for i in range(num_cnn):
            if i == 0:
                shortcut = conv_block(x, channel*4, (1,1), strides=2, activation=None)
                x = conv_block(x, channel, (1,1), strides=2)
                x = conv_block(x, channel, (3,3))
                x = conv_block(x, channel*4, (1,1), activation=None)
            else:
                shortcut = x
                x = conv_block(x, channel, (1,1))
                x = conv_block(x, channel, (3,3))
                x = conv_block(x, channel*4, (1,1), activation=None)
            
            x = keras.layers.Add()([x, shortcut])
            x = keras.layers.Activation('relu')(x)
    
    # ResNet-34
    else:
        for i in range(num_cnn):
            if block_num > 0 and i == 0: # block_num은 왜 들어갈까?
                shortcut = conv_block(x, channel, (1,1), strides=2, activation=None)
                x = conv_block(x, channel, (3,3), strides=2)
                x = conv_block(x, channel, (3,3), activation=None)
            else:
                shortcut = x
                x = conv_block(x, channel, (3,3))
                x = conv_block(x, channel, (3,3), activation=None)
            
            x = keras.layers.Add()([x, shortcut])
            x = keras.layers.Activation('relu')(x)

    return x

In [10]:
def build_resnet(input_shape=(32,32,3),
                 num_cnn_list=[3,4,6,3],
                 channel_list=[64,128,256,512],
                 num_classes=10, 
                 resnet_50=True, 
                 activation='softmax',
                 name='ResNet_50'):
    
    assert len(num_cnn_list) == len(channel_list) # 모델을 만들기 전에 config list들이 같은 길이인지 확인합니다.
    
    input_layer = keras.layers.Input(shape=input_shape)  # input layer를 만들어둡니다.
    
    # first layer
    x = conv_block(input_layer, 64, (7,7), strides=2)
    x = keras.layers.MaxPool2D(pool_size=(2,2), strides=2)(x)
    
    # config list들의 길이만큼 반복해서 Residual block 블록을 생성합니다.
    for block_num, (num_cnn, channel) in enumerate(zip(num_cnn_list, channel_list)):
        x = build_resnet_block(x, 
                               num_cnn=num_cnn, 
                               channel=channel, 
                               block_num=block_num, 
                               resnet_50=resnet_50)
        
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dense(num_classes, 
                           activation=activation, 
                           kernel_initializer='he_normal'
                          )(x)
    
    model = keras.Model(inputs=input_layer, outputs=x, name=name)
    
    return model

# 2) ResNet-34, ResNet-50 Complete Model
## ResNet 34

In [11]:
resnet_34 = build_resnet(input_shape=(32, 32,3), resnet_50=False)
resnet_34.summary()

Model: "ResNet_50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 16, 16, 64)   9472        input_2[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 16, 16, 64)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 8, 8, 64)     0           activation_1[0][0]               
__________________________________________________________________________________________

## ResNet 50

In [13]:
resnet_50 = build_resnet(input_shape=(32, 32,3), resnet_50=True)
resnet_50.summary()

Model: "ResNet_50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_37 (Conv2D)              (None, 16, 16, 64)   9472        input_3[0][0]                    
__________________________________________________________________________________________________
activation_34 (Activation)      (None, 16, 16, 64)   0           conv2d_37[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_2 (MaxPooling2D)  (None, 8, 8, 64)     0           activation_34[0][0]              
__________________________________________________________________________________________

# 3) 일반 네트워크(plain network) 만들기

In [14]:
# resnet block에서 shortcut을 제외한 plain block
def build_plain_block(input_layer, 
                      num_cnn=3, 
                      channel=64, 
                      block_num=0, 
                      is_50=True):
    
    x = input_layer
    
    # plain-50
    if is_50:
        for i in range(num_cnn):
            if block_num > 0 and i == 0:
                x = conv_block(x, channel, (1,1), strides=2)
                x = conv_block(x, channel, (3,3))
                x = conv_block(x, channel*4, (1,1))
            else:
                x = conv_block(x, channel, (1,1))
                x = conv_block(x, channel, (3,3))
                x = conv_block(x, channel*4, (1,1))
    
    # plain-34
    else:
        for i in range(num_cnn):
            if block_num > 0 and i == 0:
                x = conv_block(x, channel, (3,3), strides=2)
                x = conv_block(x, channel, (3,3))
            else:
                x = conv_block(x, channel, (3,3))
                x = conv_block(x, channel, (3,3))

    return x

In [15]:
def build_plainnet(input_shape=(32,32,3), 
                   num_cnn_list=[3,4,6,3],
                   channel_list=[64,128,256,512],
                   num_classes=10,
                   is_50=True, 
                   activation='softmax',
                   name='Plain_50'):
    
    assert len(num_cnn_list) == len(channel_list)
    
    input_layer = keras.layers.Input(shape=input_shape)
    
    # first layer
    x = conv_block(input_layer, 64, (7,7), strides=2)
    x = keras.layers.MaxPool2D(pool_size=(2,2), strides=2)(x)
    
    # config list들의 길이만큼 반복해서 plain block 블록을 생성합니다.
    for block_num, (num_cnn, channel) in enumerate(zip(num_cnn_list, channel_list)):
        x = build_plain_block(x, 
                              num_cnn=num_cnn, 
                              channel=channel,
                              block_num=block_num,
                              is_50=is_50)
    
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dense(num_classes, 
                           activation=activation,
                           kernel_initializer='he_normal'
                          )(x)
    
    model = keras.Model(inputs=input_layer, outputs=x, name=name)
    
    return model

In [16]:
plain_34 = build_plainnet(input_shape=(224,224,3), is_50=False, name='Plain_34')
plain_34.summary()

Model: "Plain_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_90 (Conv2D)           (None, 112, 112, 64)      9472      
_________________________________________________________________
activation_83 (Activation)   (None, 112, 112, 64)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_91 (Conv2D)           (None, 56, 56, 64)        36928     
_________________________________________________________________
activation_84 (Activation)   (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_92 (Conv2D)           (None, 56, 56, 64)        369

In [17]:
plain_50 = build_plainnet(input_shape=(224,224,3), is_50=True)
plain_50.summary()

Model: "Plain_50"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_123 (Conv2D)          (None, 112, 112, 64)      9472      
_________________________________________________________________
activation_116 (Activation)  (None, 112, 112, 64)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_124 (Conv2D)          (None, 56, 56, 64)        4160      
_________________________________________________________________
activation_117 (Activation)  (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_125 (Conv2D)          (None, 56, 56, 64)        369

# 4) ResNet-50 vs Plain-50 또는 ResNet-34 vs Plain-34

In [18]:
BATCH_SIZE = 16
EPOCH = 10

In [25]:
ds_train = apply_normalize_on_dataset(ds_train, batch_size=BATCH_SIZE)
ds_test = apply_normalize_on_dataset(ds_test, batch_size=BATCH_SIZE)

In [27]:
resnet_34 = build_resnet(input_shape=(224, 224, 3), 
                         num_classes=1, 
                         resnet_50=False, 
                         activation='sigmoid', 
                         name='ResNet_34')

In [28]:
resnet_34.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(lr=0.001),
    metrics=['accuracy'],
)

history_resnet_34 = resnet_34.fit(
    ds_train,
    steps_per_epoch=int(ds_info.splits['train[:80%]'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info.splits['train[80%:]'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1,
    use_multiprocessing=True,
)

Epoch 1/10
1163/1163 [==============================] - 279s 213ms/step - loss: 1737.6726 - accuracy: 0.4978 - val_loss: 0.6934 - val_accuracy: 0.4920
Epoch 2/10
1163/1163 [==============================] - 266s 229ms/step - loss: 0.6934 - accuracy: 0.5063 - val_loss: 0.6923 - val_accuracy: 0.5255
Epoch 3/10
1163/1163 [==============================] - 231s 198ms/step - loss: 0.6925 - accuracy: 0.5135 - val_loss: 0.6911 - val_accuracy: 0.5235
Epoch 4/10
1163/1163 [==============================] - 231s 198ms/step - loss: 0.6958 - accuracy: 0.5122 - val_loss: 0.6936 - val_accuracy: 0.4866
Epoch 5/10
1163/1163 [==============================] - 230s 198ms/step - loss: 0.6932 - accuracy: 0.5029 - val_loss: 0.6866 - val_accuracy: 0.5813
Epoch 6/10
1163/1163 [==============================] - 230s 198ms/step - loss: 0.6945 - accuracy: 0.5300 - val_loss: 0.6943 - val_accuracy: 0.4875
Epoch 7/10
1163/1163 [==============================] - 231s 198ms/step - loss: 0.6931 - accuracy: 0.5065 - v

KeyboardInterrupt: 

아직 미완성입니다

빠른 시간내에 완성하겠습니다